In [32]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.preprocessing import OneHotEncoder
import cv2
from sklearn.metrics import confusion_matrix, roc_curve
import seaborn as sns
import datetime
import io
import os
import pandas as pd
import random
from google.colab import files
from PIL import Image
import albumentations as A
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, InputLayer, BatchNormalization, Input, Dropout, RandomFlip, RandomRotation, Resizing, Rescaling
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, FalsePositives, FalseNegatives, TruePositives, TrueNegatives, Precision, Recall, AUC, binary_accuracy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import Callback, CSVLogger, EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers  import L2, L1
from tensorboard.plugins.hparams import api as hp
from google.colab import drive
from sklearn.model_selection import train_test_split
import itertools



In [33]:
df = pd.read_csv("/content/retail_store_inventory.csv")

In [34]:
df.head()

,Date,Store ID,Product ID,Category,Region,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Weather Condition,Holiday/Promotion,Competitor Pricing,Seasonality
0,2022-01-01,S001,P0001,Groceries,North,231,127,55,135.47,33.50,20,Rainy,0,29.69,Autumn
1,2022-01-01,S001,P0002,Toys,South,204,150,66,144.04,63.01,20,Sunny,0,66.16,Autumn
2,2022-01-01,S001,P0003,Toys,West,102,65,51,74.02,27.99,10,Sunny,1,31.32,Summer
3,2022-01-01,S001,P0004,Toys,North,469,61,164,62.18,32.72,10,Cloudy,1,34.74,Autumn
4,2022-01-01,S001,P0005,Electronics,East,166,14,135,9.26,73.64,0,Sunny,0,68.95,Summer


In [35]:
print(df['Category'].unique())
print("\n")
print(df['Region'].unique())
print("\n")
print(df['Weather Condition'].unique())
print("\n")
print(df['Seasonality'].unique())

['Groceries' 'Toys' 'Electronics' 'Furniture' 'Clothing']


['North' 'South' 'West' 'East']


['Rainy' 'Sunny' 'Cloudy' 'Snowy']


['Autumn' 'Summer' 'Winter' 'Spring']


In [36]:
one_hot = OneHotEncoder(handle_unknown='ignore' , sparse_output = False).set_output(transform ="pandas")
categorie = one_hot.fit_transform(df[['Category']])
region = one_hot.fit_transform(df[['Region']])
weather_condition = one_hot.fit_transform(df[['Weather Condition']])
seasonality = one_hot.fit_transform(df[['Seasonality']])

In [37]:
df=pd.concat([df ,categorie ] , axis = 1).drop(columns = 'Category')
df=pd.concat([df ,region ] , axis = 1).drop(columns = 'Region')
df=pd.concat([df ,weather_condition ] , axis = 1).drop(columns = 'Weather Condition')
df=pd.concat([df ,seasonality ] , axis = 1).drop(columns = 'Seasonality')

In [38]:
df.head()

,Date,Store ID,Product ID,Inventory Level,Units Sold,Units Ordered,Demand Forecast,Price,Discount,Holiday/Promotion,...,Region_South,Region_West,Weather Condition_Cloudy,Weather Condition_Rainy,Weather Condition_Snowy,Weather Condition_Sunny,Seasonality_Autumn,Seasonality_Spring,Seasonality_Summer,Seasonality_Winter
0,2022-01-01,S001,P0001,231,127,55,135.47,33.50,20,0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,2022-01-01,S001,P0002,204,150,66,144.04,63.01,20,0,...,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
2,2022-01-01,S001,P0003,102,65,51,74.02,27.99,10,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,2022-01-01,S001,P0004,469,61,164,62.18,32.72,10,1,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2022-01-01,S001,P0005,166,14,135,9.26,73.64,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [39]:
df.isnull().sum()

,0
Date,0
Store ID,0
Product ID,0
Inventory Level,0
Units Sold,0
Units Ordered,0
Demand Forecast,0
Price,0
Discount,0
Holiday/Promotion,0


In [40]:
df.head().keys()

Index(['Date', 'Store ID', 'Product ID', 'Inventory Level', 'Units Sold',
       'Units Ordered', 'Demand Forecast', 'Price', 'Discount',
       'Holiday/Promotion', 'Competitor Pricing', 'Category_Clothing',
       'Category_Electronics', 'Category_Furniture', 'Category_Groceries',
       'Category_Toys', 'Region_East', 'Region_North', 'Region_South',
       'Region_West', 'Weather Condition_Cloudy', 'Weather Condition_Rainy',
       'Weather Condition_Snowy', 'Weather Condition_Sunny',
       'Seasonality_Autumn', 'Seasonality_Spring', 'Seasonality_Summer',
       'Seasonality_Winter'],
      dtype='object')

In [41]:
x = df.drop(columns = ['Date' , 'Store ID' ,'Product ID'])
y = df['Price']

In [42]:
x = pd.DataFrame(x)
y = pd.DataFrame(y)

x_mean = x.mean()
x_std = x.std()
x_normalization = (x-x_mean) / x_std

y_mean = y.mean()
y_std = y.std()
y_normalization = (y - y_mean) / y_std

In [43]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [44]:
x_train.shape , y_train.shape

((58480, 25), (58480, 1))

In [45]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, input_shape=(x_train.shape[1],), activation="relu"),
    tf.keras.layers.Dense(20, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation="relu"),
    tf.keras.layers.Dense(1, activation=None)
])

model.compile(
    loss=tf.keras.losses.MeanAbsoluteError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=["mae"]
)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [46]:
model.fit(x_train, y_train, validation_split=0.2,  epochs=50,  batch_size =32)

Epoch 1/50


Exception ignored in: <function _xla_gc_callback at 0x7c0293bd37f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


1462/1462 ━━━━━━━━━━━━━━━━━━━━ 51s 3ms/step - loss: 5.1261 - mae: 5.1261 - val_loss: 12.3946 - val_mae: 12.3946
Epoch 2/50
1462/1462 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1.6386 - mae: 1.6386 - val_loss: 11.8340 - val_mae: 11.8340
Epoch 3/50
1462/1462 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.3414 - mae: 1.3414 - val_loss: 8.9933 - val_mae: 8.9933
Epoch 4/50
1462/1462 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 1.2619 - mae: 1.2619 - val_loss: 10.8115 - val_mae: 10.8115
Epoch 5/50
1462/1462 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - loss: 1.2943 - mae: 1.2943 - val_loss: 12.2187 - val_mae: 12.2187
Epoch 6/50
1462/1462 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 1.1237 - mae: 1.1237 - val_loss: 10.6517 - val_mae: 10.6517
Epoch 7/50
1462/1462 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.9464 - mae: 0.9464 - val_loss: 10.9141 - val_mae: 10.9141
Epoch 8/50
1462/1462 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.8531 - mae: 0.8531 - val_loss: 10.2370 - val_mae: 10.2370
Epoch 9/50
1462/1462 ━━━━━━━━━━━━━━━

In [47]:
y_pred =model.predict(x_test)


457/457 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step


In [56]:
y_pred = pd.DataFrame(y_pred)
y_pred

,0
0,33.552406
1,16.070118
2,17.336723
3,80.285797
4,9.154741
...,...
14615,56.683601
14616,59.407986
14617,33.246033
14618,35.982296


In [51]:
y_test

,Price
66127,40.47
43485,18.65
34304,20.33
34317,98.65
38631,10.04
...,...
8251,69.41
50745,72.55
50634,39.94
23941,43.33
